# **Model Evaluation**
Bu bölümde, proje kapsamında eğitilen modellerin performansları, güçlü ve zayıf yönleri, sınıf dengesizliği etkisi ve nihai model seçiminin gerekçeleri sistematik olarak değerlendirilmiştir. Amaç, kullanılan yöntemlerin karar alma sürecine etkisini açıklamak ve sonraki aşamalara ışık tutmaktır.

## 1. Genel Bakış

Home Credit Default Risk veri seti, ciddi ölçüde **dengesiz (imbalanced)** bir hedef değişkenine sahiptir. Başvuranların yaklaşık %92’si kredi ödemesinde sorun yaşamazken yalnızca %8’i temerrüde düşmektedir. Bu dengesiz yapı, model performansını özellikle **recall**, **precision** ve **f1-score** üzerinde belirgin şekilde etkilemektedir.

Bu nedenle model değerlendirme aşamasında yalnızca **accuracy** değil; **ROC-AUC**, **confusion matrix**, **class-wise recall** ve **threshold ayarı** gibi daha uygun metrikler kullanılmıştır.

---

## 2. Baseline Model: Logistic Regression

İlk olarak kurulan lojistik regresyon modeli, bu proje için bir **baseline** olarak değerlendirilmiştir. Orijinal veri ile eğitilen modelin:

- **ROC-AUC skoru ~0.75**  
- Minority class için **recall ≈ 0.01–0.02**  
- Balanced class weight uygulandığında recall yükselse de precision ciddi şekilde düşmüş  
- SMOTE uygulandığında ise modelin genel performansı belirgin biçimde kötüleşmiştir.

Bu sonuçlar, lojistik regresyonun karmaşık yapıda ve çok boyutlu bu veri setinde sınırlı kaldığını, özellikle de **non-lineer ilişki yakalama kabiliyetinin düşük olduğunu** göstermektedir.

---

## 3. Alternatif Model: XGBoost

Lojistik regresyonun sınırlı performansı nedeniyle ikinci aşamada **XGBoost** modeli eğitilmiştir. Bu model:

- Feature engineering sonrasında oluşturulan 217 değişkeni etkin şekilde işleyebilmiştir.
- Non-lineer ilişkileri, etkileşimleri ve önem dereceleri farklı olan değişkenleri daha iyi yakalamıştır.

XGBoost’un temel performans sonuçları:

- **ROC-AUC ≈ 0.7678**  
- Threshold optimizasyonundan önce minority class recall düşüktü  
- Threshold 0.36 civarına çekildiğinde:
  - Recall anlamlı şekilde artmış  
  - Precision düşmesine rağmen f1-score dengelenmiştir  
  - Genel doğruluk beklenen şekilde azalmış olsa da model risk tespitinde daha etkili hale gelmiştir.

Bu, kredi riski tahmini gibi **false negative maliyetinin yüksek olduğu** senaryolarda daha tutarlı bir davranıştır.

---

## 4. Threshold Optimizasyonu

Default threshold (0.50) kredi riski gibi dengesiz veri setlerinde ideal değildir. Bu nedenle ROC eğrisi üzerinden optimal bir eşik belirlendi.

Seçilen eşik değeri:
- **Threshold ≈ 0.3618**

Bu eşikle model:
- Daha fazla riskli müşteriyi doğru tespit etmektedir,
- "kaçırılan riskli başvuru" sayısını azaltmaktadır,
- Bu nedenle gerçek dünya kullanımına daha uygun bir çıktı üretmektedir.

---

## 5. Confusion Matrix Değerlendirmesi

Seçilen nihai eşik ile:
- **Gerçek riskli sınıfın %72’si tespit edilebilmiştir (recall↑)**  
- Yanlış pozitifler (false positives) artmıştır, bu beklendik bir durumdur.  
- Modelin amacı göz önünde bulundurulduğunda bu değişim **kabul edilebilir ve faydalıdır** çünkü temerrüde düşme ihtimali olan bireyleri gözden kaçırmamak önceliklidir.

---

## 6. Feature Importance Analizi

XGBoost'un ürettiği önem sıralamasına göre modelde en belirleyici değişkenler:

1. **EXT_SOURCE_2**  
2. **EXT_SOURCE_3**  
3. **EXT_SOURCE_1**  
4. Eğitim seviyesi ve gelir türü kategorileri  
5. **CREDIT_TERM**, **AMT_GOODS_PRICE**, **AGE**  
6. Bazı demografik ve kredi başvuru davranışlarıyla ilgili değişkenler

Bu sonuçlar literatür ile paraleldir. Özellikle **EXT_SOURCE değişkenleri**, dış skorlama sisteminden geldiği için güçlü prediktörlerdir.

---

## 7. Nihai Model Seçimi

Yapılan denemeler sonucunda:

### En dengeli performansa sahip model: XGBoost
- ROC-AUC en yüksek değer  
- Recall optimizasyonu yapılabilir  
- Dengesiz veriyle başa çıkma becerisi daha başarılı  
- Karmaşık ilişkileri daha iyi modelleyen bir yapıya sahip  

Bu nedenle proje kapsamındaki **nihai model XGBoost olarak belirlenmiştir**.

---

## 8. Modelin Sınırlılıkları

- One-hot encoding sonrası çok yüksek boyut nedeniyle bazı ilişkiler basitleştirilmiş olabilir.  
- Hızlı çalışması için sınırlı hiperparametre optimizasyonu yapılmıştır.  
- Hesaplama gücü sınırlı olduğu için RandomizedSearch / GridSearch tam kapsamlı uygulanamamıştır.  
- Veri setinde yüksek missing-rate olan birçok değişken tamamen kaldırılmıştır; bu, modelin potansiyelini kısıtlayabilir.

---

## 9. Sonraki Adımlar (Gelecek Çalışmalar)

Projenin devamında aşağıdaki iyileştirmeler uygulanabilir:

### 1. Gelişmiş Hiperparametre Araması
GPU destekli RandomizedSearch ya da Optuna ile hızlandırılmış arama yapılabilir.

### 2. Daha İyi Dengesizlik Stratejileri
- SMOTE yerine **SMOTE-Tomek**, **SMOTE-ENN**, **ADASYN**,  
- Undersampling + Ensemble modelleri,  
- “Focal Loss” gibi yöntemler denenebilir.

### 3. Feature Selection
Shap değerleri ile daha sağlam bir feature importance analizi yapılabilir.

### 4. Model Setinin Birleştirilmesi
Stacking veya voting classifier ile XGBoost + Logistic Regression kombinasyonu test edilebilir.

### 5. Şirket Senaryosu Simülasyonu
Kredi karar süreçleri için farklı threshold senaryolarının maliyet analizi yapılabilir.


# **Genel Değerlendirme**

Bu proje kapsamında elde edilen sonuçlar, veri setinin dengesiz yapısına rağmen modelin kredi temerrüdünü anlamlı biçimde öngörebildiğini göstermektedir. XGBoost, baseline modele göre belirgin bir gelişme sağlamıştır ve threshold optimizasyonu ile gerçek dünyadaki risk yönetimi senaryolarına daha uygun hale gelmiştir.

Bu nedenle, oluşturulan model hem akademik açıdan yeterli hem de uygulamaya yönelik güçlü bir çözüm sunmaktadır.